In [ ]:
x = """# ADR-001: Architecture & Roadmap for BJVN Parallel Viewer\n\nTitle: Architectural Decision Record — Buddhist Journal of Viet Nam (BJVN) Parallel Viewer App  \nStatus: Accepted  \nDate: 2025-09-02  \nOwner: Aaron Solomon  \nRepo: BJVN-parallel-viewer\n\n—\n\n## 1. Context\n\nWe are building a parallel viewer for the Buddhist Journal of Viet Nam (BJVN) archival project. The goal is to allow readers to:\n\n1. View scanned page images.\n2. View Vietnamese OCR and English translations side-by-side.\n3. Navigate easily across pages and sections.\n4. In the future, propose corrections to OCR and translations.\n5. Maintain stable data contracts so we can evolve the backend independently.\n\nThis ADR defines the architecture, stack, models, phasing plan, and future backend options.\n\n—\n\n## 2. Goals\n\n### Functional Goals\n- Serve bilingual text + page scans in a polished, intuitive interface.\n- Ensure stable anchors (aid) per span for future features.\n- Phase in editing capabilities without blocking initial deployment.\n- Support multiple backend strategies for Phase II without frontend churn.\n\n### Non-Functional Goals\n- Rapid Phase I deployment with minimal complexity.\n- High-quality, accessible UI out of the box.\n- Preserve flexibility for future editorial workflows.\n- Enable migration to CMS-driven backends if needed.\n\n—\n\n## 3. Architecture Overview\n\n### 3.1 Phase I — Viewer-Only App\nStack:\n- Backend: FastAPI\n- Templating: Jinja2\n- UI Frameworks: TailwindCSS + DaisyUI (for polished, composable components)\n- Interactive Enhancements: HTMX (lightweight dynamic interactivity)\n- Page Viewer: OpenSeadragon (deep zoom, crisp scans)\n- Data Source: Static JSON bundles per issue (pre-generated)\n- Hosting: Fly.io / Render (containerized, single service)\n- Storage: Local files for JSON + images (no DB)\n\nCapabilities:\n- Browse documents → /doc/{doc_id}.\n- Pan/zoom scanned pages.\n- View Vietnamese + English spans in parallel.\n- External form integration for suggesting corrections.\n- JSON contract frozen to enable seamless backend replacement.\n\n### 3.2 Phase II — Editable Backend\nEnhancements:\n- Add paragraph-level editing + revision history.\n- Preserve stable aid anchors for tracking changes.\n- Implement moderation workflows.\n\nBackend Options:\n- Option A: SQLite + Custom Revisions (default path)\n    - Keep FastAPI/Jinja UI.\n    - Add spans + revisions tables.\n    - Store immutable edit history.\n    - Backup via Litestream → S3/B2.\n\n- Option B: Integrate Wagtail as Editorial Backend\n    - Use Wagtail for content storage, revisions, and workflows.\n    - Keep the existing FastAPI viewer OR migrate templates into Wagtail.\n    - Wagtail exposes /api/doc/{doc_id}.json to serve the same bundle schema.\n\n- Option C: Hybrid CMS + FastAPI\n    - Run Wagtail separately at admin.bjvn.org.\n    - Editors manage spans; FastAPI viewer fetches bundles via REST.\n\n—\n\n## 4. Data Model & Contracts\n\n### 4.1 JSON Bundle Schema (frozen)\nThis schema is the public API contract between backend(s) and frontend:\n\njson\n{\n  \"doc_id\": \"ang_v_1956_issue5\",\n  \"title\": \"Phật Giáo Việt Nam — ANG.V\",\n  \"pages\": [\n    { \"page\": 1, \"image\": \"/images/angv_p001.jpg\" },\n    { \"page\": 2, \"image\": \"/images/angv_p002.jpg\" }\n  ],\n  \"sections\": [\n    {\n      \"sid\": \"s1\",\n      \"title_vi\": \"NHỮNG NGÀY HOAN-HỶ\",\n      \"title_en\": \"Joyful Days\",\n      \"spans\": [\n        {\n          \"aid\": \"angv:s1:p1\",\n          \"page\": 4,\n          \"vi\": \"...\",\n          \"en\": \"...\"\n        }\n      ]\n    }\n  ]\n}\n\n\n#### Key Notes:\n- aid = stable paragraph anchor ({doc_id}:{section_id}:{paragraph_id}).\n- pages[] = ordered scans, allows OpenSeadragon linking.\n- spans[] = ordered bilingual paragraph units.\n- Metadata (like translation notes) may be added later, but core keys must remain stable.\n\n### 4.2 SQLite Schema (Phase II, Option A)\n\nsql\n-- Table of documents\nCREATE TABLE documents (\n    id TEXT PRIMARY KEY,\n    title TEXT\n);\n\n-- Latest state of each span\nCREATE TABLE spans (\n    aid TEXT PRIMARY KEY,\n    doc_id TEXT,\n    page INTEGER,\n    vi TEXT,\n    en TEXT,\n    version INTEGER\n);\n\n-- Immutable revision history\nCREATE TABLE revisions (\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    aid TEXT,\n    lang TEXT,\n    prev_version INTEGER,\n    new_version INTEGER,\n    old_text TEXT,\n    new_text TEXT,\n    created_at TEXT,\n    user_hint TEXT\n);\n\n\nThis supports:\n- Conflict detection (version compare before commit).\n- Full audit trail.\n- Easy syncing to Wagtail or other CMS if needed.\n\n### 4.3 Wagtail Model Mapping (Phase II, Option B)\n\npython\nclass Document(Page):\n    doc_id = models.SlugField(unique=True)\n    title = models.CharField(max_length=255)\n\nclass Span(models.Model):\n    document = models.ForeignKey(Document, related_name=\"spans\", on_delete=models.CASCADE)\n    aid = models.CharField(max_length=100, unique=True)\n    page = models.IntegerField()\n    vi = models.TextField(blank=True)\n    en = models.TextField(blank=True)\n\nclass SpanRevision(models.Model):\n    span = models.ForeignKey(Span, related_name=\"revisions\", on_delete=models.CASCADE)\n    lang = models.CharField(max_length=2, choices=[(\"vi\",\"Vietnamese\"),(\"en\",\"English\")])\n    prev_version = models.IntegerField()\n    new_version = models.IntegerField()\n    old_text = models.TextField()\n    new_text = models.TextField()\n    created_at = models.DateTimeField(auto_now_add=True)\n    user_hint = models.CharField(max_length=200, blank=True)\n\n\nWagtail automatically provides:\n- Built-in revisions per page/field.\n- Editorial dashboards and workflows.\n- REST API endpoints for exporting bundles.\n\n—\n\n## 5. Phasing Plan\n\n### Phase I — MVP (Viewer-Only)\n| Feature | Status | Stack |\n|———|––––|—––|\n| View scans | ✅ | OpenSeadragon |\n| View VN + EN | ✅ | FastAPI + Jinja |\n| Navigate pages | ✅ | JSON bundle contract |\n| Suggest corrections | ✅ | External form (Formspree / Netlify) |\n| Deploy public viewer | 🚀 | Fly.io / Render |\n\n### Phase II — Enhanced Editing\n| Capability | Option A: SQLite | Option B: Wagtail |\n|————|——————|––––––––––|\n| Inline editing | ✅ | ✅ |\n| Revision history | ✅ | ✅ |\n| Moderation workflow | ⚠️ manual | ✅ built-in |\n| REST API for bundles | ✅ custom | ✅ built-in |\n| Admin dashboard | ❌ | ✅ |\n| External integrations | Manual | Rich Wagtail ecosystem |\n\n—\n\n## 6. Risks & Mitigations\n\n| Risk | Impact | Mitigation |\n|——|––––|————|\n| Lock-in to custom edit code | Medium | Keep JSON schema stable; Wagtail can reproduce API |\n| OCR alignment errors | High | Use AI-assisted alignment later; maintain aid anchors |\n| Phase II complexity | High | Start with smallest viable SQLite + HTMX edit flow |\n| Hosting costs | Low | Start on Fly.io free-tier; consider Netlify for static assets |\n\n—\n\n## 7. Decision Summary\n\n- Phase I: Viewer-first, minimal FastAPI + JSON bundles + Tailwind + DaisyUI.\n- Editing Deferred: Suggest-only via external form; no DB yet.\n- Freeze Bundle Contract: Frontend consumes a stable JSON API that any backend can emit.\n- Phase II Options:\n    1. SQLite module (lightweight, easy)\n    2. Wagtail CMS (full editorial power)\n    3. Hybrid FastAPI + Wagtail REST integration.\n\n—\n\n## 8. Next Steps\n\n1. Finalize repository skeleton.\n2. Implement /api/doc/{doc_id}.json in FastAPI now.\n3. Ship Phase I viewer on Fly.io / Render.\n4. Set up external correction form (Formspree or Netlify Forms).\n5. Plan Phase II editorial backend in Q2–Q3 2026.\n\n—\n\n## 9. Appendix\n\n- Repo name: BJVN-parallel-viewer\n- Bundle source: data/*.json\n- Images directory: images/*.jpg\n- API contract: /api/doc/{doc_id}.json\n- Preferred backend roadmap: FastAPI → SQLite → optional Wagtail\n\n—\n\nOutcome:\nThis ADR establishes a clear contract-first design and phased roadmap for the BJVN Parallel Viewer. By separating the frontend viewer from backend persistence concerns, we preserve optionality and allow a smooth upgrade path to SQLite or Wagtail while delivering a polished MVP quickly"""

In [2]:
from tnh_scholar.utils.file_utils import write_str_to_file

from pathlib import Path

p = Path("ADR-001")
write_str_to_file(p, x, overwrite=True)